In [0]:
CREATE OR REFRESH STREAMING TABLE movies_bronze
COMMENT "Cette table est une table bronze connecté au dossier csv des films pokemons"
AS(
  SELECT `Unnamed: 0` as row_id, imdbID, Title, Year, Type
  FROM cloud_files('gs://de-01-data-ingestion/csv/', 'csv', 
    map(
      "cloudFiles.inferColumnTypes", "true",
      'header', 'true',
      'delimiter', ',',        
      'quote', '"',            
      'escape', '"',
      'multiLine', 'false',
      "cloudFiles.schemaLocation", "gs://de-01-data-ingestion/csv/schema/movies_schema"
    )
  )
)

In [0]:
CREATE OR REFRESH STREAMING TABLE movies_silver(
CONSTRAINT year_not_null EXPECT (Year IS NOT NULL) ON VIOLATION DROP ROW,
CONSTRAINT Title_not_null EXPECT (Title IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "Cette table est une table silver connecté à la table bronze movies_bronze"
AS(
  SELECT *
  FROM STREAM(LIVE.movies_bronze)
)

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW movies_by_year_gold 
COMMENT "Cette table est une table gold connecté à la table silver movies_silver et affiche le nombre total de films par année"
AS
SELECT Year, COUNT(*) AS Total
FROM LIVE.movies_silver
GROUP BY Year

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW movies_by_type_gold 
COMMENT "Cette table est une table gold connecté à la table silver movies_silver et affiche le nombre total de films par Type"
AS
SELECT Type, COUNT(*) AS Total
FROM LIVE.movies_silver
GROUP BY Type